In [2]:
## Convert RGB to Grayscale
import cv2
import os
def convert_rgb_to_grayscale(sr_path, save_path):
    for img_name in os.listdir(sr_path):
        img_path = os.path.join(sr_path, img_name)
        img = cv2.imread(img_path)
        print("shape of image: ", img.shape)
        if img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            cv2.imwrite(os.path.join(save_path, img_name), img)
            img = cv2.imread(img_path)
            print("shape of image: ", img.shape)

            
sr_path = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages\EVAL_02_test_PL1_5e_FL2_V2"
save_path = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages\dummy"
convert_rgb_to_grayscale(sr_path, save_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_PL1_5e_FL2_V2'

In [3]:
drone_dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"

dir_dict = {}
for folder in os.listdir(drone_dir):
    if folder[0] =="E" and folder not in [ 'EVAL_04_test_A_BicubicUpscale', "EVAL_04_test_A_BicubicUpscale"]:
        sr_img_dir = os.path.join(drone_dir, folder)
        dir_dict[folder] = sr_img_dir
dir_dict

{'EVAL_02_test_PL1_5e_FL2': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_PL1_5e_FL2',
 'EVAL_02_test_PL1_FL1': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_PL1_FL1',
 'EVAL_02_test_PL1_FL2': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_PL1_FL2',
 'EVAL_02_test_PL2_FL2': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_PL2_FL2',
 'EVAL_02_test_sobel': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_sobel',
 'EVAL_02_test_sobel_5e': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\validation_datasets\\DroneImages\\EVAL_02_test_sobel_5e',
 'EVAL_03_test_PL1_5e_FL2': 'C:\\SaiVinay\\SproutsAI\\GitHub_\\ir_srgan_otsr\\model_validation\\

In [ ]:
result_paths = {
    ""
}

In [4]:

sr_img_dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages\EVAL_02_test_PL1_5e_FL2_V2"
%cd C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\src_ir_imgs

### Convert RGB to Grayscale
# convert_rgb_to_grayscale(sr_img_dir)

from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import utils
import os



def get_validation_metrice():
    for gt_imgs in ["DroneImgs", "FLIR"]:
        if gt_imgs == "DroneImgs":
            _dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
            gt_img_dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\DroneImages\test_images"
            print("Getting metrics for Drone Images")
        if gt_imgs == "FLIR":
            _dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
            gt_img_dir = r"C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset\test_images"
            print("Getting metrics for FLIR Images")
        dir_dict = {}
        for folder in os.listdir(_dir):
            if folder[0] =="E" and folder not in [ 'EVAL_04_test_A_BicubicUpscale', "EVAL_04_test_B_BicubicUpscale"]:
                if "05" in folder:
                    sr_img_dir = os.path.join(_dir,folder, "KernelGAN_resized")
                    dir_dict[folder] = sr_img_dir
                    continue
                sr_img_dir = os.path.join(_dir, folder)
                dir_dict[folder] = sr_img_dir
        for result_name, sr_img_dir in dir_dict.items():
            print("##############################################")
            print(f"\n Getting metrics for {result_name} \n")
            print("##############################################")
            gt_list_dir = os.listdir(gt_img_dir)
            sr_list_dir = os.listdir(sr_img_dir)

            gt_paths = [os.path.join(gt_img_dir, i) for i in gt_list_dir]
            sr_paths = [os.path.join(sr_img_dir, i) for i in sr_list_dir]
            avg_psnr, avg_ssim, max_psnr, max_ssim = 0, 0, 0, 0
            for i in range(len(gt_paths)):
                try:
                    gt_path = gt_paths[i]
                    sr_path = sr_paths[i]
                    # convert_rgb_to_grayscale(sr_path)
                    gt_img = np.array(Image.open(gt_path))
                    sr_img = np.array(Image.open(sr_path))
                    if len(sr_img.shape) == 3:
                        sr_img = np.array(Image.open(sr_path).convert("L"))
                    # print(f"Size of : {gt_img.shape} and {sr_img.shape}")
                    
                    crop_size = 4
                    gt_img = gt_img / 255.
                    sr_img = sr_img / 255.
                    cropped_sr_img = sr_img[crop_size:-crop_size, crop_size:-crop_size]
                    cropped_gt_img = gt_img[crop_size:-crop_size, crop_size:-crop_size]
                    psnr = utils.calculate_psnr(cropped_sr_img * 255, cropped_gt_img * 255)
                    ssim = utils.calculate_ssim(cropped_sr_img * 255, cropped_gt_img * 255)
                    if psnr > max_psnr:
                        max_psnr = psnr
                    if ssim > max_ssim:
                        max_ssim = ssim
                    avg_psnr += psnr
                    avg_ssim += ssim
                except Exception as e:
                    # print(f"Error: {e}")
                    continue

            avg_psnr = avg_psnr / len(gt_paths)
            avg_ssim = avg_ssim / len(gt_paths)
            print(f"Average PSNR: {avg_psnr}")
            print(f"Average SSIM: {avg_ssim}")
            print(f"Max PSNR: {max_psnr}")
            print(f"Max SSIM: {max_ssim}")

get_validation_metrice()

C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\src_ir_imgs
Getting metrics for Drone Images
##############################################

 Getting metrics for EVAL_02_test_PL1_5e_FL2 

##############################################
Average PSNR: 18.179939750768845
Average SSIM: 0.7622308074126374
Max PSNR: 23.565524896705064
Max SSIM: 0.9262430958246988
##############################################

 Getting metrics for EVAL_02_test_PL1_FL1 

##############################################
Average PSNR: 18.17660233620849
Average SSIM: 0.6929455983513892
Max PSNR: 23.825399159886942
Max SSIM: 0.8731962147665914
##############################################

 Getting metrics for EVAL_02_test_PL1_FL2 

##############################################
Average PSNR: 19.669995155386054
Average SSIM: 0.7556552172043474
Max PSNR: 25.71067880083874
Max SSIM: 0.8839642299097544
##############################################

 Getting metrics for EVAL_02_test_PL2_FL2 

#############################

In [5]:
import torch

print(torch.__version__)

2.2.1+cu118
